# SQLAlchemy

- Set up the model in data.model
- Create the database and migrate to newer versions using alembic
- Connect using SQLAlchemy

In [24]:
import pandas as pd
from sqlalchemy import select
# We usually keep a single blank row between standard and 
# custom modules
from data.model import Shape, Trip, Stop, StopTime
from visualisations.maps import routemap, add_positions
from data.data import Session, path, engine

We can fetch trips using a simple select query

In [2]:
with Session() as session:
    stmt = select(Trip).limit(10)
    trips = session.execute(stmt).scalars().all()

trips

[Trip(route_id='2447_S886', service_id='294', id='1000095', trip_headsign='Mount View HS'),
 Trip(route_id='2447_S871', service_id='294', id='1000097', trip_headsign='Mount View HS'),
 Trip(route_id='2447_S879', service_id='294', id='1000103', trip_headsign='Cessnock PS'),
 Trip(route_id='2454_8615', service_id='294', id='1000377', trip_headsign='Blaxland Station'),
 Trip(route_id='2454_8311', service_id='294', id='1000381', trip_headsign='Warrimoo Primary'),
 Trip(route_id='2454_8237', service_id='294', id='1000385', trip_headsign='Blaxland High School'),
 Trip(route_id='2454_8530', service_id='248', id='1000387', trip_headsign='Mt Riverview'),
 Trip(route_id='2454_8716', service_id='294', id='1000403', trip_headsign='Blue Mtns Hospital'),
 Trip(route_id='2454_8728', service_id='294', id='1001298', trip_headsign='Wentworth Falls Stn'),
 Trip(route_id='2446_880', service_id='56', id='1004848', trip_headsign='Campbelltown')]

And we can inspect individual values

In [3]:
trips[0].route_id

'2447_S886'

If we need all records in a dataframe, just use pd.read_sql

In [4]:
trips_dataframe = pd.read_sql(select(Trip), engine)

Sqlalchemy allows us to write succinct queries in a pythonic way. Notice
that in the query below, we didn't have to specific how the join is
defined. This is because it's set up once in the model, and can then be
easily reused in future queries.

In [5]:
with Session() as session:
    stmt = select(Shape).join(Trip).where(Trip.id == 1948209)
    shape = session.execute(stmt).scalars().all()

shape

[Shape(id='151217', lat=-33.874502, lon=150.9289512, sequence=1, dist_traveled=0.0),
 Shape(id='151217', lat=-33.8745039, lon=150.9289457, sequence=2, dist_traveled=6.0),
 Shape(id='151217', lat=-33.8742754, lon=150.9279578, sequence=3, dist_traveled=101.0),
 Shape(id='151217', lat=-33.874062, lon=150.9270657, sequence=4, dist_traveled=187.0),
 Shape(id='151217', lat=-33.8739998, lon=150.9267969, sequence=5, dist_traveled=212.0),
 Shape(id='151217', lat=-33.8739998, lon=150.9267969, sequence=6, dist_traveled=227.0),
 Shape(id='151217', lat=-33.8737898, lon=150.925894, sequence=7, dist_traveled=314.0),
 Shape(id='151217', lat=-33.873751, lon=150.9257328, sequence=8, dist_traveled=329.0),
 Shape(id='151217', lat=-33.8732564, lon=150.9258096, sequence=9, dist_traveled=384.0),
 Shape(id='151217', lat=-33.872959, lon=150.9258168, sequence=10, dist_traveled=417.0),
 Shape(id='151217', lat=-33.8726064, lon=150.9257605, sequence=11, dist_traveled=457.0),
 Shape(id='151217', lat=-33.8719352, lo

Because we have a primary key on trip, we can query even more simply
using session.get

In [6]:
with Session() as session:
    shape = session.get(Trip, 1948209).shape

In [7]:
session = Session()

In [8]:
trip = session.get(Trip, 1948209)

In [9]:
trip.stop_times

[StopTime(trip_id='1948209', stop_id='2165100', arrival_time='15:10:00', departure_time='15:10:00', stop_sequence=1),
 StopTime(trip_id='1948209', stop_id='2165101', arrival_time='15:11:00', departure_time='15:11:00', stop_sequence=2),
 StopTime(trip_id='1948209', stop_id='2165138', arrival_time='15:15:00', departure_time='15:15:00', stop_sequence=3),
 StopTime(trip_id='1948209', stop_id='2165139', arrival_time='15:17:00', departure_time='15:17:00', stop_sequence=4),
 StopTime(trip_id='1948209', stop_id='2165140', arrival_time='15:18:00', departure_time='15:18:00', stop_sequence=5),
 StopTime(trip_id='1948209', stop_id='2165207', arrival_time='15:18:00', departure_time='15:18:00', stop_sequence=6),
 StopTime(trip_id='1948209', stop_id='2165140', arrival_time='15:18:00', departure_time='15:18:00', stop_sequence=7),
 StopTime(trip_id='1948209', stop_id='2165141', arrival_time='15:19:00', departure_time='15:19:00', stop_sequence=8),
 StopTime(trip_id='1948209', stop_id='2165142', arrival_

In [23]:
with Session() as session:
    stmt = select(Stop.lat, Stop.lon).join(StopTime).join(Trip).where(Trip.id == 1948209)
    df_stops = pd.DataFrame(session.execute(stmt).all())

pd.DataFrame(df_stops)

,lat,lon
0,-33.874556,150.928933
1,-33.874052,150.926752
2,-33.869601,150.924547
3,-33.868870,150.921433
4,-33.868485,150.919739
...,...,...
72,-33.858202,150.941371
73,-33.858521,150.942590
74,-33.859268,150.945538
75,-33.860533,150.945686


In [11]:
df_route = pd.DataFrame.from_records(
    [(s.id, s.lat, s.lon) for s in shape], 
    columns=['shape_id', 'lat', 'lon']
)

In [27]:
fig = routemap(df_route)
add_positions(fig, df_stops)